In [1]:
from sklearn.datasets import load_boston
import statsmodels.api as sm

In [2]:
boston = load_boston()

dfX = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy = pd.DataFrame(boston.target, columns=['MEDV'])
df = pd.concat([dfX, dfy], axis=1)

model = sm.OLS.from_formula("MEDV ~ " + " + ".join(boston.feature_names), data = df)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 16 Aug 2021   Prob (F-statistic):          6.72e-135
Time:                        14:22:19   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

**warning**을 보면 조건수(condition number)가 15100으로 너무 크고 다중공선성(multicollinearity) 문제가 있을 수 있다고 나타난다.

## 조건수(condition number)

조건수란 공분산 행렬 $X^TX$의 가장 큰 고윳값과 가장 작은 고윳값의 비율을 뜻한다.

$$\text{condition number} = \dfrac{\lambda_{\text{max}}}{\lambda_{\text{min}}}$$

실제로 위의 boston 데이터를 이용해 공분산 행렬을 만들고 고유분해를 진행한 뒤 고윳값의 최댓값과 최솟값을 비교하면 다음과 같이 매우 큰 차이를 보이는 것을 확인할 수 있다.

In [27]:
eigen = np.linalg.eig(dfX.values.T @ dfX.values)[0]
eigen

array([1.58386796e+08, 1.18747372e+07, 4.17002244e+05, 1.61644573e+05,
       2.52697480e+04, 1.47629635e+04, 8.18396001e+03, 6.07326738e+03,
       4.23577535e+03, 6.06399504e+02, 3.27412564e+02, 3.04157837e+01,
       2.19326965e+00])

In [28]:
np.min(eigen), np.max(eigen)

(2.1932696529903897, 158386795.65291503)

조건수가 가장 작은 경우는 공분산 행렬이 단위행렬인 경우이다. 이 때의 조건수 값은 1이다.

$$cond(I) = 1$$

$X$가 다음과 같은 단위행렬일 때 

In [1]:
X = np.eye(4)
X

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

$y$가 1-벡터인 경우 $X^{-1}$을 구해서 가중치 벡터 $w$를 구할 수 있다.

In [6]:
X

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [3]:
y

array([1., 1., 1., 1.])

In [2]:
y = np.ones(4)

w = np.linalg.solve(X, y)
w

array([1., 1., 1., 1.])

만약 여기서 $X$에 약간의 오차가 있었다면 어땠을까?

In [44]:
X_error = X + 0.0001 * np.eye(4)
X_error

array([[1.0001, 0.    , 0.    , 0.    ],
       [0.    , 1.0001, 0.    , 0.    ],
       [0.    , 0.    , 1.0001, 0.    ],
       [0.    , 0.    , 0.    , 1.0001]])

In [46]:
w_error = np.linalg.solve(X_error, y)
w_error

array([0.99990001, 0.99990001, 0.99990001, 0.99990001])

행렬 $A$에 1/10000 의 오차가 있었는데 가중치 벡터를 구한 결과 역시 유사한 정도의 오차가 발생했다.

이처럼 약간의 오차가 있는 데이터를 이용해 회귀분석 결과인 가중치 벡터를 구하면 역시 비슷한 정도의 오차가 발생한다는 것이 일반적인 사고이다. 하지만 다음과 같이 조건수가 큰 경우에 대해 가중치 벡터를 구해보자.

다음의 행렬은 4차 힐버트 행렬(Hilbert Matrix)라는 조건수가 15000이 넘는 특이한 행렬이다.

In [50]:
import scipy as sp

X = sp.linalg.hilbert(4)
X

array([[1.        , 0.5       , 0.33333333, 0.25      ],
       [0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.25      , 0.2       , 0.16666667, 0.14285714]])

In [51]:
np.linalg.cond(X)

15513.738738929662

오차가 없을 때의 연립방정식의 해

In [55]:
np.linalg.solve(X, y)

array([  -4.,   60., -180.,  140.])

1/10000 의 오차가 발생했을 때의 연립방정식의 해

In [56]:
np.linalg.solve(X + 0.0001 * np.eye(4), y)

array([ -0.58897672,  21.1225671 , -85.75912499,  78.45650825])

이처럼 조건수가 큰 경우에는 1/10000 이라는 아주 작은 오차가 주어졌음에도 불구하고 매우 큰 예측 결과를 반환하는 것을 확인할 수 있다.

> 공분산행렬 $X^TX$의 조건수가 크면 회귀분석 예측값도 오차가 커진다.

## 회귀분석과 조건수

회귀분석에서 조건수가 커지는 원인은 크게 두가지로 분류된다.

1. 변수들의 스케일 문제(스케일링으로 해결)
2. 다중공선성 문제, 독립변수간의 큰 상관관계가 있는 경우(PCA, 변수 선택으로 해결)

앞선 회귀분석에서 사용한 보스턴 데이터의 독립변수들의 통계량 중 표준편차를 확인한 결과는 다음과 같다.

각 독립변수간의 스케일이 크게 다른것을 확인할 수 있다.

In [62]:
dfX.describe().loc["std"]

CRIM         8.601545
ZN          23.322453
INDUS        6.860353
CHAS         0.253994
NOX          0.115878
RM           0.702617
AGE         28.148861
DIS          2.105710
RAD          8.707259
TAX        168.537116
PTRATIO      2.164946
B           91.294864
LSTAT        7.141062
Name: std, dtype: float64

조건수와 회귀분석의 관계를 파악하기 위해 일부러 TAX 의 스케일을 높게 조정했다.

그 결과 회귀분석의 성능지표인 R-squared 값이 0.333으로 떨어진 것을 확인할 수 있다.

In [63]:
dfX2 = dfX.copy()
dfX2["TAX"] *= 1e13
df2 = pd.concat([dfX2, dfy], axis=1)

model2 = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     83.42
Date:                Mon, 16 Aug 2021   Prob (F-statistic):           8.42e-44
Time:                        14:46:01   Log-Likelihood:                -1737.9
No. Observations:                 506   AIC:                             3484.
Df Residuals:                     502   BIC:                             3501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0038      0.000     -8.554      0.0

이번에는 `scale()`명령을 이용해 스케일링을 진행한 뒤 회귀분석을 진행한 결과 조건수 값이 매우 작게 떨어진 것을 확인할 수 있다.

In [64]:
feature_names = list(boston.feature_names)
feature_names.remove("CHAS") 
feature_names = ["scale({})".format(name) for name in feature_names] + ["CHAS"]
model3 = sm.OLS.from_formula("MEDV ~ " + "+".join(feature_names), data=df2)
result3 = model3.fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 16 Aug 2021   Prob (F-statistic):          6.72e-135
Time:                        14:47:12   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         22.3470      0.219    101.